# 2. Data wrangling   
### 2.1 Data overviews  
### 2.2 Importing related libs & modules  
### 2.3 Loading data  
- Gun Possession: Total of gun licensees per <span style="color:red">year</span>  
- Gun Deaths: Total gun deaths (homicide, sucide...) per <span style="color:red">year</span>  
- Mass Shooting: Number of mass shooting cases with shooter age, gender, mental health, employment by <span style="color:red">year, states</span>  
- Unemployment rate: Unemployment rate by <span style="color:red">years, states</span>  
### 2.4 Explore the data  
- Data distribution & missing values  
- Numeric features  
- Category features   
### 2.5 Target variables  
### 2.6 Save data  
### 2.7 Sumary  


## 2.1 Data Overviews  
### input data  
index: Year 2009 - 2018  
Dependent variable (y): Total mass shooting victims      
Independent variables (Xi): population, state_population, age, gender, total_licensees, unemployment_rate, employees, mental_health_issues
  
### What to do  
Load, transform 

## 2.2 Import libs & modules  
### geoplot for geodetic display

In [1]:
import numpy as np
import pandas as pd
import os

## 2.3 Loading data  
### DF: GunDeaths_2009_2018; GunPossession_1986_2018; Mass_Shooting; Employment;   
Additional data sorting by year.

In [2]:
GunDeaths_2009_2018  = pd.read_csv('../data/USA_Crimes/GunDeaths_2009_2018.csv').set_index('Year')
GunPossession_1986_2018 = pd.read_csv('../data/USA_Crimes/GunPossession_1986_2018.csv', index_col=0)

In [3]:
GunPossession_1986_2018 = GunPossession_1986_2018[['Total Licensees ','Licensed Business Entities']]
GunPossession_1986_2018.columns = ['total_licensees ','licensed_business_entities']
GunPossession_1986_2018['year'] = GunPossession_1986_2018.index
# GunPossession_1986_2018.info()

In [4]:
GunDeaths_2009_2018 = GunDeaths_2009_2018[['Population','Total gun deaths','Total children and teen gun deaths']]
GunDeaths_2009_2018.columns = ['population','total_gun_deaths','total_children_teen_gun_deaths']
GunDeaths_2009_2018['year'] = GunDeaths_2009_2018.index
# GunDeaths_2009_2018.columns 

### **Combine data**
#### Data: $\color{cyan}{\text{Year, Polpulation, Total licenses, Gun deaths, age, gender.}}$
#### Additional data = GunDeaths_2009_2018 + GunPossession_1986_2018 + Employment by year & states

Yearly employment info by states

In [5]:
Employment_header = ['FIPS Code', 'State and area', 'Year', 'Civilian non-institutional population', 
  'Civilian labor force/Total', 'Civilian labor force/Percent of population', 
  'Civilian labor force/Employment/Total', 'Civilian labor force/Employment/Percent of population',
  'Civilian labor force/Unemployment/Total','Civilian labor force/Unemployment/Rate']

In [122]:
# Employment = pd.read_excel('../data/USA_Crimes/staadata.xlsx', header=0)
Employment=pd.read_excel(
     os.path.join("../data/USA_Crimes/", "staadata.xlsx"),
     engine='openpyxl', header=None, names= Employment_header,skiprows=8
).set_index('Year')

In [123]:
Employment = Employment[['State and area', 'Civilian non-institutional population', 
  'Civilian labor force/Total','Civilian labor force/Unemployment/Rate']]
Employment.columns = ['state','state_population','state_labor_force','unemployment_rate']
Employment['year'] = Employment.index

In [16]:
Employment.head(2)

,state,state_population,state_labor_force,unemployment_rate,year
Year,,,,,
1976,Alabama,2632667,1501284,6.8,1976
1976,Alaska,239917,163570,7.6,1976


In [20]:
Employment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332 entries, 1976 to 2019
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   state              2332 non-null   object 
 1   state_population   2332 non-null   int64  
 2   state_labor_force  2332 non-null   int64  
 3   unemployment_rate  2332 non-null   float64
 4   year               2332 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 109.3+ KB


#### DF = GunDeaths_2009_2018 + GunPossession_1986_2018 + Employment + Mass shooting by year & states  
Mass shooting for total_victims, employees, mental health, age, gender, employed_at

In [124]:
Mass_Shooting = pd.read_csv('../data/USA_Crimes/USMassShooting19662019(cleaned).csv', parse_dates=True, index_col='Date')

In [125]:
MS_column_drop = ['S#','Title','Area','Incident Area','Open/Close Location','Target','Cause','Summary',
  'Shooter status','No. of shooter/suspect','Race','Latitude','Longitude']
Mass_Shooting.sort_index(inplace=True)
Mass_Shooting.drop(MS_column_drop, axis=1, inplace=True)

In [126]:
Mass_Shooting['date'] = pd.to_datetime(Mass_Shooting.index)
Mass_Shooting['year'] = Mass_Shooting['date'].dt.year
# Mass_Shooting['month'] = Mass_Shooting['date'].dt.month
# Mass_Shooting['monthday'] = Mass_Shooting['date'].dt.day
# Mass_Shooting['weekday'] = Mass_Shooting['date'].dt.weekday

In [127]:
#get states from locations
def get_state(txt):
    val = txt.split(", ")[-1]            
    return val

Mass_Shooting['state'] = Mass_Shooting['Location'].apply(lambda x : get_state(x))

In [128]:
Mass_Shooting.drop('Location',axis=1,inplace=True)
Mass_Shooting.columns = ['fatalities', 'injured', 'total_victims',
       'policeman_killed', 'age', 'employeed(Y/N)', 'employed_at',
       'mental_health_issues', 'gender', 'date', 'year', 'state']

In [129]:
print(len(np.sort(Mass_Shooting.state.unique())),np.sort(Mass_Shooting.state.unique()))
print(len(np.sort(Employment.state.unique())),np.sort(Employment.state.unique()))

53 [' Virginia' 'Alabama' 'Alaska' 'Arizona' 'Arkansas' 'CA' 'California'
 'Colorado' 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho'
 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Lousiana'
 'Maine' 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi'
 'Missouri' 'Montana' 'NV' 'Nebraska' 'Nevada' 'New Jersey' 'New Mexico'
 'New York' 'North Carolina' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'South Carolina' 'South Dakota' 'Tennessee' 'Texas' 'Texas ' 'Utah'
 'Vermont' 'Virginia' 'Washington' 'Washington D.C.' 'West Virginia'
 'Wisconsin' 'Wyoming']
53 ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'District of Columbia' 'Florida' 'Georgia'
 'Hawaii' 'Idaho' 'Illinois' 'Indiana' 'Iowa' 'Kansas' 'Kentucky'
 'Los Angeles County' 'Louisiana' 'Maine' 'Maryland' 'Massachusetts'
 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nebraska'
 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New Yo

In [130]:
name_revise = {'District of Columbia': 'Washington D.C.','NV': 'Nevada','New York city': 'New York',
'CA': 'California','Lousiana': 'Louisiana','Los Angeles County': 'California',
'Texas ': 'Texas',' Virginia': 'Virginia'}

# Employment["state"] = Employment["state"].apply(lambda x: name_revise.get(x))
# Mass_Shooting["state"] = Mass_Shooting["state"].apply(lambda x: str(name_revise.get(x)))

Mass_Shooting["state"].replace(name_revise, inplace=True)
Employment["state"].replace(name_revise, inplace=True)


In [131]:
print(len(np.sort(Mass_Shooting.state.unique())),np.sort(Mass_Shooting.state.unique()))
print(len(np.sort(Employment.state.unique())),np.sort(Employment.state.unique()))

48 ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Jersey' 'New Mexico' 'New York' 'North Carolina'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'South Carolina' 'South Dakota'
 'Tennessee' 'Texas' 'Utah' 'Vermont' 'Virginia' 'Washington'
 'Washington D.C.' 'West Virginia' 'Wisconsin' 'Wyoming']
51 ['Alabama' 'Alaska' 'Arizona' 'Arkansas' 'California' 'Colorado'
 'Connecticut' 'Delaware' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Iowa' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Maryland'
 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana'
 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey' 'New Mexico' 'New York'
 'North Carolina' 'North Dakota' 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania'
 'Rhode I

In [132]:
emp_state = np.sort(Employment.state.unique())
mas_state = np.sort(Mass_Shooting.state.unique())
print(set(emp_state)^set(mas_state))

{'North Dakota', 'New Hampshire', 'Rhode Island'}


In [133]:
print(Mass_Shooting.shape,GunPossession_1986_2018.shape,GunDeaths_2009_2018.shape,Employment.shape)

(339, 12) (33, 3) (10, 4) (2332, 5)


### Start merging data

In [134]:
df = pd.merge(Mass_Shooting,GunPossession_1986_2018,on=['year'],how='left')

In [135]:
df = pd.merge(df,GunDeaths_2009_2018,on=['year'],how='left')

In [136]:
df = pd.merge(df,Employment,on=['year','state'],how='left')

In [137]:
df.set_index('date',inplace=True)

#### Filter data within 2009-2018 of which the mass shooting data is fully covered

In [ ]:
df.drop(df[(df.year<2009) | (df.year>2018)].index,inplace=True)

In [139]:
print(Mass_Shooting.shape,GunPossession_1986_2018.shape,GunDeaths_2009_2018.shape,Employment.shape,df.shape)

(339, 12) (33, 3) (10, 4) (2332, 5) (244, 19)


#### Few attempts of exploring final data

In [142]:
df['total_victims'][df.year==2013].sum()

133

In [141]:
df['total_victims'][(df.year==2018) & (df.state=='Texas')].sum()

23

## 2.4 Explore data  
### Data transforming  

In [144]:
df.isnull().sum()

fatalities                        0
injured                           0
total_victims                     0
policeman_killed                  0
age                               0
employeed(Y/N)                    0
employed_at                       0
mental_health_issues              0
gender                            0
year                              0
state                             0
total_licensees                   0
licensed_business_entities        0
population                        0
total_gun_deaths                  0
total_children_teen_gun_deaths    0
state_population                  0
state_labor_force                 0
unemployment_rate                 0
dtype: int64

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 244 entries, 2009-03-10 to 2018-11-19
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   fatalities                      244 non-null    int64  
 1   injured                         244 non-null    int64  
 2   total_victims                   244 non-null    int64  
 3   policeman_killed                244 non-null    int64  
 4   age                             244 non-null    object 
 5   employeed(Y/N)                  244 non-null    int64  
 6   employed_at                     244 non-null    object 
 7   mental_health_issues            244 non-null    object 
 8   gender                          244 non-null    object 
 9   year                            244 non-null    int64  
 10  state                           244 non-null    object 
 11  total_licensees                 244 non-null    object 
 12  licensed_business

##  2.5 Target variables  
index: Year 2009 - 2018  
Dependent variable (y): total_victims  
Independent variables (Xi): population, state_population, age, gender, total_licensees, unemployment_rate, employees, mental_health_issues

## 2.6 Save data  

In [145]:
df.to_csv('../data/Clean_data/MassShooting2009_2018_final.csv')

## 2.7 Summary  

- The final data is a time series which has 19 features and 244 not-null observations.  
- Keys features are total_victims, population, state_population, age, gender, total_licensees, unemployment_rate, employees, mental_health_issues.  
- The data need to be dived deeper, processed & statistical analyzed in order to form hypothesis & determine them to get as much insights as possible. Those futher steps are in the next EDA stage.